In [246]:
import pandas as pd


blood_df = pd.read_csv('blood_tests-data.csv', dtype={'id_report': 'str'})
blood_info_df = pd.read_csv('blood_tests-info.csv')

# check that all blood values are unique for each row
def components_unique():
    all_unique = blood_info_df['component_short'].value_counts().all() == 1
    if not all_unique:
        raise Exception("All blood components must be unique")
components_unique()

    
# this is source of truth. All columns/components that indicate blood values stem from this
all_components = list(blood_info_df['component_short'])


# in percent 
# margin_scale = {
#     'red': 10,
#     'orange': 15
# }


# get component short and full names. Example: WBC <-> white blood cells
def get_component(name, kind):
    if kind == 'full':
        return blood_info_df[blood_info_df['component_short'] == name]['component_full'].to_string(index=False)
    
    if kind == 'short':
        return blood_info_df[blood_info_df['component_full'] == name]['component_short'].to_string(index=False)
    


# convert all columns that indicate blood components to float type 
def components_float():
    for comp in all_components:
        blood_df[comp] = blood_df[comp].astype('float')
components_float()


# convert date column to date type
blood_df['date_donation'] = pd.to_datetime(blood_df['date_donation'])

In [253]:
# find row with information about a specific blood test component
def component_info(comp):
    return blood_info_df.loc[blood_info_df['component_short'] == comp]


# check if is abnormal and if yes return results
def check_abnormal(curr, min_v, max_v, compare=0):
    side = ''
    limit = 0
    if curr > max_v:
        limit = max_v
        side = 'upper'
        result = 'out'
    elif curr < min_v:
        limit = min_v
        side = 'lower'
        result = 'out'
    else:
        limit = compare
        result = 'in'
    return {'side': side, 'result': result, 'diff': round(abs(curr - limit), 3)}



# find all abnormal blood values 
def find_abnormal_only(df):
    
    list_rows = []
    abnormal_cols = ['id_report', 'date_donation', 'component_short', 'component_full', 'value', 'min', 'max', 'diff', 'side']
    
    for i in df.index:
        this_row = df.iloc[i]
        for comp in all_components:
            this_v = this_row[comp]
            info_row = component_info(comp)
            min_v = float(info_row['min'])
            max_v = float(info_row['max'])
            
            abnormal = check_abnormal(this_v, min_v, max_v)

            if abnormal['result'] == 'out':
                abnormal_row = {
                    'id_report': this_row['id_report'],
                    'date_donation': this_row['date_donation'],
                    'component_short': comp,
                    'component_full': info_row['component_full'].to_string(index=False),
                    'value': this_v,
                    'min': min_v,
                    'max': max_v,
                    'diff': abnormal['diff'],
                    'side': abnormal['side']
                }
                
                list_rows.append(abnormal_row)
                
    
    return pd.DataFrame(list_rows, columns=abnormal_cols)


abnormal_df = find_abnormal_only(blood_df)
abnormal_df



,id_report,date_donation,component_short,component_full,value,min,max,diff,side
0,004425,2017-04-30,NEUT,neutrophils,35.83,40.0,70.0,4.17,lower
1,004425,2017-04-30,LYMPH,lymphocytes,51.12,20.0,40.0,11.12,upper
2,010165,2017-08-13,PLT,platelets,141.00,150.0,400.0,9.00,lower
3,010165,2017-08-13,AST,aspartate aminotransferase,56.00,0.0,37.0,19.00,upper
4,010085,2018-07-08,MCH,mean corpuscular hemoglobin,31.29,26.0,31.2,0.09,upper
5,014781,2018-10-21,LYMPH,lymphocytes,43.78,20.0,40.0,3.78,upper
6,014781,2018-10-21,ALT,alanine aminotransferease,43.00,0.0,41.0,2.00,upper
7,007767,2022-05-22,ser_iron,serum iron,160.00,37.0,158.0,2.00,upper


In [299]:

# find blood values that are not abnormal, but are close to the min or max normal values
# given a margin in percentage of how close value can be. This can help monitor which blood values might need more attention 
# the smaller the percentage, the more marging you're giving 
# the bigger the percentage, the more 
# default margin percentage is 10%. It can't be higher than 50%

def in_range(curr, min_v, max_v):
    return curr >= min_v and curr <= max_v
        
    

def normal_near_limit(df, margin=10):
    list_rows = []
    near_limit_cols = ['id_report', 'date_donation', 'component_short', 'component_full', 'value', 'min', 'max', 'diff', 'side']
    
    for i in df.index:
        this_row = df.iloc[i]
        for comp in all_components:
            this_v = this_row[comp]
            info_row = component_info(comp)
            min_v = float(info_row['min'])
            max_v = float(info_row['max'])
            
            normal = check_abnormal(this_v, min_v, max_v)['result'] == 'in'
            
            if normal:
                # m = margin percentage of max - min
                # m is the value of the margin compared to min and max
                interval = max_v - min_v
                m = interval * margin / 100
                x = min_v + m 
                y = max_v - m
                
#                 print("comp: {} \nid: {} \nthis_v: {} \nmin_v: {} \nmax_v: {} \nmargin: {} \nmax_v - min_v: {} \nm: {} \nx: {} \ny: {} \n\n".format(comp, this_row['id_report'], this_v, min_v, max_v, margin, interval, m, x,  y))
                
                if in_range(this_v, min_v, x):
                    diff = x - min_v
                    side = 'lower'
                elif in_range(this_v, y, max_v):
                    diff = max_v - y
                    side = 'upper'
                else:
                    continue
                   
                
                normal_limit_row = {
                    'id_report': this_row['id_report'],
                    'date_donation': this_row['date_donation'],
                    'component_short': comp,
                    'component_full': info_row['component_full'].to_string(index=False),
                    'value': this_v,
                    'min': min_v,
                    'max': max_v,
                    'diff': diff,
                    'side': side
                }
                
                list_rows.append(normal_limit_row)
                
                
                
    return pd.DataFrame(list_rows, columns=near_limit_cols)

            
normal_near_limit_df = normal_near_limit(blood_df)
normal_near_limit(blood_df)
    
    

,id_report,date_donation,component_short,component_full,value,min,max,diff,side
0,010165,2017-08-13,MCH,mean corpuscular hemoglobin,31.06,26.0,31.2,0.52,upper
1,016348,2017-12-09,PLT,platelets,173.00,150.0,400.0,25.00,lower
2,016348,2017-12-09,LYMPH,lymphocytes,38.28,20.0,40.0,2.00,upper
3,016348,2017-12-09,ser_iron,serum iron,158.00,37.0,158.0,12.10,upper
4,010085,2018-07-08,ser_iron,serum iron,48.00,37.0,158.0,12.10,lower
5,014781,2018-10-21,PLT,platelets,157.00,150.0,400.0,25.00,lower
6,014781,2018-10-21,NEUT,neutrophils,40.38,40.0,70.0,3.00,lower
7,009771,2019-07-07,PLT,platelets,159.00,150.0,400.0,25.00,lower
8,014422,2019-10-13,LYMPH,lymphocytes,38.36,20.0,40.0,2.00,upper
9,007767,2022-05-22,chol,cholesterol,198.00,0.0,200.0,20.00,upper


In [297]:
new_df = normal_near_limit_df.groupby('side').get_group('upper')

new_df
      
# normal_near_limit_df.pivot(
#     columns='side',
#     index='date_donation',
#     values='value'
# )


,id_report,date_donation,component_short,component_full,value,min,max,diff,side
0,010165,2017-08-13,MCH,mean corpuscular hemoglobin,31.06,26.0,31.2,0.52,upper
2,016348,2017-12-09,LYMPH,lymphocytes,38.28,20.0,40.0,2.00,upper
3,016348,2017-12-09,ser_iron,serum iron,158.00,37.0,158.0,12.10,upper
8,014422,2019-10-13,LYMPH,lymphocytes,38.36,20.0,40.0,2.00,upper
9,007767,2022-05-22,chol,cholesterol,198.00,0.0,200.0,20.00,upper
